# Point of Sale(POS) Data from a Supermarket: Transactions and Cashier Operations

We will use SQL for queries and Plotly for visualization.

Importing libraries that we may use.

In [ ]:
import plotly.express as px
import numpy as np

## Task 1: Exploring the data

A good first step in exploring a database is to select the first few rows from each table.

In [ ]:
SELECT *
FROM 'pos_operator_logs.csv'
LIMIT 10

,id,Workstation_Group_ID,Workstation_ID,begin_date_time,operator_id
0,1,8,19,2019-02-13 05:37:55+00:00,269
1,2,8,18,2019-02-13 05:37:55+00:00,268
2,3,8,17,2019-02-13 05:38:43+00:00,267
3,4,1,4,2019-02-13 07:01:26+00:00,332
4,5,1,7,2019-02-13 07:01:57+00:00,10
5,6,1,7,2019-02-13 07:02:16+00:00,10
6,7,1,4,2019-02-13 07:03:54+00:00,332
7,8,1,4,2019-02-13 07:05:52+00:00,332
8,9,1,7,2019-02-13 07:32:51+00:00,114
9,10,1,7,2019-02-13 07:35:12+00:00,114


In [ ]:
SELECT *
FROM 'pos_transactions.csv'
LIMIT 10

,id,WorkstationGroupID,begin_date_time,end_date_time,OperatorID,basket_size,t_cash,t_card,amount
0,1,1,2017-12-07 14:23:23+00:00,2017-12-07 14:24:36+00:00,101,23,True,False,112.71
1,2,1,2017-12-07 14:25:09+00:00,2017-12-07 14:27:00+00:00,101,29,True,False,54.76
2,3,1,2017-12-07 14:27:28+00:00,2017-12-07 14:27:48+00:00,101,3,True,False,14.77
3,4,1,2017-12-07 14:28:04+00:00,2017-12-07 14:28:29+00:00,101,12,True,False,37.88
4,5,1,2017-12-07 14:29:40+00:00,2017-12-07 14:30:32+00:00,101,7,True,False,115.34
5,6,1,2017-12-07 14:31:03+00:00,2017-12-07 14:32:01+00:00,101,12,False,True,232.89
6,7,1,2017-12-07 14:32:25+00:00,2017-12-07 14:36:03+00:00,101,36,True,False,183.15
7,8,1,2017-12-07 14:36:32+00:00,2017-12-07 14:37:15+00:00,101,10,True,False,39.68
8,9,1,2017-12-07 14:37:31+00:00,2017-12-07 14:38:07+00:00,101,15,True,False,44.53
9,10,1,2017-12-07 14:38:21+00:00,2017-12-07 14:39:12+00:00,101,15,False,True,42.73


## Task 2: Do more people make transactions by card or by cash?

Here we look to understand if people prefer to make transactions from their card or cash.

In [ ]:
SELECT
SUM(t_cash::numeric) AS trans_cash,
SUM(t_card::numeric) AS trans_card
FROM 'pos_transactions.csv'

,trans_cash,trans_card
0,84487.0,78246.0


More transactions were done using cash than with card.
Let's visualise this using pie plot

In [ ]:
px.pie(df1,values= df1.iloc[0],names=['Trans_cash','Trans_card'],
       title='RATIO OF TRANSACTION OF CASH AND CARD',
      color_discrete_sequence=px.colors.sequential.Jet)

## Task 3: Do people spend more per transaction when using cash or card?

Similarly, knowing how much people spend per transaction using different payment methods is helpful for deciding on security arrangments, and marketing to encourage shoppers to use different payment methods.


In [ ]:
SELECT
AVG(CASE WHEN t_cash == 'True' THEN amount END) as avg_amount_cash,
AVG(CASE WHEN t_card == 'True' THEN amount END) AS avg_amount_card
FROM 'pos_transactions.csv'

,avg_amount_cash,avg_amount_card
0,59.622365,87.560859


The average amount using card is more than when using cash. This makes sense as purchases in high amount is more convenient using card.

Let's visualise this with a pie plot. Hover the plot to see more details

In [ ]:
fig = px.pie(df2,values=df2.iloc[0],names=df2.columns,hover_data=df2.values,title='RATIO AMOUNT FROM CARD AND CASH', color_discrete_sequence=px.colors.sequential.Jet)
fig.show()

## Task 4: Exploring Sunday Trading

"Regulations were introduced in Poland in 2018 to banned shopping on some Sundays. It was generally two Sundays per month in 2018 and three Sundays per month in 2019. Supermarkets have reacted by extending the working hours on Fridays and Saturdays, while customers had to adapt to the changes in opening hours. **The dataset contains two working Sundays (24 February, 31 March) and two non-working Sundays (17 February, 7 April)."**

The above statement is from the data source(link is at the top of this workspace).

In [ ]:
SELECT
DATE_PART('weeks', end_date_time::DATE) AS week_num,
end_date_time::DATE AS end_date_time
FROM 'pos_transactions.csv'
WHERE end_date_time::DATE IN ('2019-02-17','2019-02-24','2019-03-31','2019-04-07')
	AND EXTRACT(YEAR from end_date_time::DATE) >= '2019'
GROUP BY end_date_time::DATE

,week_num,end_date_time
0,8,2019-02-24 00:00:00+00:00
1,13,2019-03-31 00:00:00+00:00


The last two dates in my query do not appear and rightly so, because they were non-working Sundays.

So the Weeks with with working Sundays are 8 and 13. And the weeks with non-working Sundays are 7 and 14. Again, this information, I got it from the data source.

Let's explore the data further to see if I have the full week data.

## Task 5: Analysis of daily trends

Understanding which days are busiest is important for deciding how many staff to schedule shifts for, as well as helping to plan financially.

In [ ]:
SELECT

DATE_PART('weeks', end_date_time::DATE) as week_num,
end_date_time::DATE as end_date,
avg(amount) as average_daily_amount,
median(amount) as median_daily_amount,
sum(amount) as total_daily_amount

FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM end_date_time) = '2019'

GROUP BY end_date_time::DATE
ORDER BY week_num

,week_num,end_date,average_daily_amount,median_daily_amount,total_daily_amount
0,7,2019-02-13 00:00:00+00:00,59.788963,37.580,189112.49
1,7,2019-02-14 00:00:00+00:00,68.629076,46.285,302242.45
2,7,2019-02-15 00:00:00+00:00,79.164584,50.900,319112.44
3,7,2019-02-16 00:00:00+00:00,86.866736,57.600,405580.79
4,8,2019-02-23 00:00:00+00:00,84.591933,56.370,369412.97
5,8,2019-02-18 00:00:00+00:00,59.062168,37.420,193133.29
6,8,2019-02-19 00:00:00+00:00,56.278486,37.530,170242.42
7,8,2019-02-21 00:00:00+00:00,68.363725,45.865,298612.75
8,8,2019-02-22 00:00:00+00:00,74.910175,49.060,330428.78
9,8,2019-02-24 00:00:00+00:00,63.341634,37.660,143025.41


Let's visualise this and see what we have

In [ ]:
for i in ['average_daily_amount','median_daily_amount','total_daily_amount']:
    fig = px.bar(df3,x='end_date', y=i,color='week_num',title=f'RELATIONSHIP BETWEEN {i.upper()} AND DAY OF WEEK', labels = {
        'end_date':'day of week'
    },  color_discrete_sequence=px.colors.sequential.Jet
                )
    fig.show()

We see that across the three plots that the three peaks occurs in Thursday,Friday and Saturday, that means, most activities occurs between thursday and saturday. Sunday has low daily amount purchases. It would make sense to schedule more staffs for the three aforementioned days.

This's an interesting pattern/trend.

Let's build on the previous analysis, adding the average and median `basket_size` per day in 2019. This will inform us on how the number of items people buy changes at different times.

I've included day in the query, just to make the visuals more concise.

In [ ]:
SELECT

	DATE_PART('weeks', end_date_time::DATE) as week_num,
	DATE_PART('dayofweek', end_date_time::DATE) as day,
	end_date_time::DATE as end_date,
	avg(amount) as average_daily_amount,
	median(amount) as median_daily_amount,
	sum(amount) as total_daily_amount,
	avg(basket_size) as avg_basket_size,
	median(basket_size) as median_basket_size

FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM end_date_time) = '2019'

GROUP BY end_date_time::DATE
ORDER BY week_num

,week_num,day,end_date,average_daily_amount,median_daily_amount,total_daily_amount,avg_basket_size,median_basket_size
0,7,3,2019-02-13 00:00:00+00:00,59.788963,37.580,189112.49,12.915903,9.0
1,7,4,2019-02-14 00:00:00+00:00,68.629076,46.285,302242.45,14.172116,10.0
2,7,5,2019-02-15 00:00:00+00:00,79.164584,50.900,319112.44,17.184818,12.0
3,7,6,2019-02-16 00:00:00+00:00,86.866736,57.600,405580.79,19.733348,14.0
4,8,6,2019-02-23 00:00:00+00:00,84.591933,56.370,369412.97,19.813144,14.0
5,8,1,2019-02-18 00:00:00+00:00,59.062168,37.420,193133.29,13.810092,10.0
6,8,2,2019-02-19 00:00:00+00:00,56.278486,37.530,170242.42,13.432727,9.0
7,8,4,2019-02-21 00:00:00+00:00,68.363725,45.865,298612.75,16.488324,12.0
8,8,5,2019-02-22 00:00:00+00:00,74.910175,49.060,330428.78,17.736568,12.0
9,8,0,2019-02-24 00:00:00+00:00,63.341634,37.660,143025.41,15.169176,10.0


Before moving to next task, let's visualise the newly added analysis using bar plot

In [ ]:
for i in ['avg_basket_size','median_basket_size']:
    fig = px.bar(df4, x='end_date',y=i,title=f'RELATIONSHIP BETWEEN {i.upper()} AND DAY OF THE WEEK',
                labels = {
                    'end_date':'day of the week'
                }, color='day',  color_discrete_sequence=px.colors.sequential.Jet)
    fig.show()

Sunday is regarded as day 0 in our plot and Saturday, day 6. Now it is obvious and easy to spot the three peaks as associated with Thursday, Friday and Saturday.

## Task 6: Should the supermarket open on Sundays?

Remember, working sundays were in week 8 and 13, while non working Sundays were in week 7 and 14.

Let's explore our dataset a little more. For weeks with working Sundays, We will see the total amount made each day and the two weeks.

In [ ]:
SELECT

	DATE_PART('dayofweek',end_date_time::DATE) AS dayofweek,
	sum(amount) as total_amount
FROM 'pos_transactions.csv'
WHERE dayofweek IN (4,5,6,0,1,2,3) AND DATE_PART('week',end_date_time::DATE) IN (8,13)
GROUP BY DATE_PART('dayofweek',end_date_time::DATE)

,dayofweek,total_amount
0,0,312723.89
1,1,193133.29
2,2,170242.42
3,3,181630.10
4,4,617900.30
5,5,670582.53
6,6,723521.64


Let's visualise the trend using a line plot. We will see the same pattern but this time more clearly. The peaks of transaction occurred on Thursdays, Saturdays and Fridays.

In [ ]:

name_label = ['Sunday','Monday','Tuesday','Wednesday','Thurday','Friday','Saturday']
figure = px.line(df5, 'dayofweek','total_amount',title='TREND OF AMOUNT ACROSS DAYS OF WEEK',text=name_label)
figure.update_traces(textposition="bottom center",
                    line_color='white', textfont={'color':'white'})
figure.update_xaxes(showgrid=False,zeroline=False)
figure.update_yaxes(gridcolor='gray')
figure.update_layout(
    plot_bgcolor="#0932e8"
)

figure.show()

Now let's query and see the total amount made in the three categories of interest: Sunday, Thursday-Saturday, Mon-Weds. By visualising the result of our query, we see that there were low purchases on sundays relative to the others. This is expected as the supermarket didn't open on some Sundays.

In [ ]:
SELECT
	SUM(amount) AS daily_thur_fri_sat,
	(SELECT
	SUM(amount) AS total_daily_amount
FROM 'pos_transactions.csv'
WHERE DATE_PART('dayofweek',end_date_time::DATE) in (0)
	) AS sunday,
(SELECT
	SUM(amount) AS total_daily_amount
FROM 'pos_transactions.csv'
WHERE DATE_PART('dayofweek',end_date_time::DATE) in (1,2,3)
 ) AS daily_total_mon_tues_weds,
FROM 'pos_transactions.csv'
WHERE DATE_PART('dayofweek',end_date_time::DATE) in (4,5,6)

,daily_thur_fri_sat,sunday,daily_total_mon_tues_weds
0,6.955165e+06,933108.9,3970181.36


In [ ]:
fig = px.bar(df6.iloc[0],title='TOTAL AMOUNT ACROSS THE THREE CATEGORIES', labels={
    'value':'Amount', 'index':'Category'
})
fig.update_traces(
    marker_color='#0932e8'
)
fig.update_layout(
    plot_bgcolor="#cfe4ff",
    showlegend=False
)

The two Sundays I see are significantly lower in volume of transactions than all other days.
Also, Thursdays, Fridays and Saturdays are approximately 25% higher in number of transactions and 40% higher in sales revenue than Mondays, Tuesdays and Wednesdays.

Does it make sense to open stores on Sundays?

The full week data is available only for week 8 and week 14. Let's compare these two.


In [ ]:
SELECT

DATE_PART('week',end_date_time::DATE) AS week_num,
SUM(amount) AS Amount

FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM end_date_time) = '2019' AND DATE_PART('week',end_date_time::DATE) IN (8,14)
GROUP BY week_num

,week_num,Amount
0,8,1686485.72
1,14,1690989.87


In [ ]:
print(f'The difference in amount made between the two weeks is {np.abs(df7.loc[0,"Amount"]-df7.loc[1,"Amount"])}')

The difference in amount made between the two weeks is 4504.150000018766


This does not look like a big difference, however, to answer the question if it makes sense to open the stores on Sundays, other factors have to be considered, like, is the operational cost for Sundays lesser than what is made?

Looking at the table below, we see that the stores had 18 employees working on Sunday 24 February and 20 employees working on Sunday 31 March.
I would look into the hours per shift and the cost per hour to calcute the average savings per Sunday, as well as estimate all other costs of keeping stores open and operational (e.g. energy costs). I would also consider if the stocks will be affected in terms of life span, etc.

These estimates will provide the basis to decide if it is sensible to open the stores on Sundays.

In [ ]:
SELECT
(begin_date_time::DATE) as working_day,
COUNT(DISTINCT(operator_id)),
(CASE
	WHEN DATE_PART('month',begin_date_time::DATE) == 1 THEN 'January'
	WHEN DATE_PART('month',begin_date_time::DATE) == 2 THEN 'Febraury'
	WHEN DATE_PART('month',begin_date_time::DATE) == 3 THEN 'March'
	WHEN DATE_PART('month',begin_date_time::DATE) == 4 THEN 'April'
END	)
 as month,
(CASE
	WHEN DATE_PART('dayofweek',begin_date_time::DATE) == 0 THEN 'Sunday'
 	WHEN DATE_PART('dayofweek',begin_date_time::DATE) == 1 THEN 'Monday'
 	WHEN DATE_PART('dayofweek',begin_date_time::DATE) == 2 THEN 'Tuesday'
 	WHEN DATE_PART('dayofweek',begin_date_time::DATE) == 3 THEN 'Wednesday'
 	WHEN DATE_PART('dayofweek',begin_date_time::DATE) == 4 THEN 'Thursday'
	WHEN DATE_PART('dayofweek',begin_date_time::DATE) == 5 THEN 'Friday'
	WHEN DATE_PART('dayofweek',begin_date_time::DATE) == 6 THEN 'Saturday'
END ) AS dayofweek

FROM 'pos_operator_logs.csv'

WHERE EXTRACT(YEAR FROM begin_date_time) >= '2019'

GROUP BY working_day

,working_day,count(DISTINCT operator_id),month,dayofweek
0,2019-02-13 00:00:00+00:00,20,Febraury,Wednesday
1,2019-02-14 00:00:00+00:00,26,Febraury,Thursday
2,2019-02-15 00:00:00+00:00,24,Febraury,Friday
3,2019-02-16 00:00:00+00:00,28,Febraury,Saturday
4,2019-02-18 00:00:00+00:00,20,Febraury,Monday
5,2019-02-19 00:00:00+00:00,21,Febraury,Tuesday
6,2019-02-20 00:00:00+00:00,19,Febraury,Wednesday
7,2019-02-21 00:00:00+00:00,27,Febraury,Thursday
8,2019-02-22 00:00:00+00:00,26,Febraury,Friday
9,2019-02-23 00:00:00+00:00,25,Febraury,Saturday
